In [17]:
# German Energy Analysis - Data Cleaning
# Purpose: Transform raw data into analysis-ready dataset

import pandas as pd
import numpy as np
from datetime import datetime

#Load raw data
df = pd.read_csv('../data/raw/energy_data_raw.csv')

print(f"Starting with {df.shape[0]} rows and {df.shape[1]} columns")

Starting with 50401 rows and 300 columns


In [18]:
# Convert to datetime
df['utc_timestamp'] = pd.to_datetime(df['utc_timestamp'], errors='coerce')

# Check for failed conversions
failed = df['utc_timestamp'].isna().sum()
if failed > 0:
    print(f"⚠️ {failed} timestamps failed to convert")

# Set as index and sort
df = df.set_index('utc_timestamp').sort_index()

# Sanity check
print("Timestamp converted successfully")
print(f"Date range: {df.index.min()} → {df.index.max()}")


Timestamp converted successfully
Date range: 2014-12-31 23:00:00+00:00 → 2020-09-30 23:00:00+00:00


In [20]:
# Analyze missing value patterns
print("\n--- Missing Value Analysis ---")
for col in df.columns:
    missing_count = df[col].isnull().sum()
    if missing_count > 0:
        missing_pct = (missing_count / len(df)) * 100
        print(f"{col}: {missing_count} missing ({missing_pct:.2f}%)")


--- Missing Value Analysis ---
AT_load_actual_entsoe_transparency: 1 missing (0.00%)
AT_load_forecast_entsoe_transparency: 1 missing (0.00%)
AT_price_day_ahead: 17556 missing (34.83%)
AT_solar_generation_actual: 62 missing (0.12%)
AT_wind_onshore_generation_actual: 49 missing (0.10%)
BE_load_actual_entsoe_transparency: 1 missing (0.00%)
BE_load_forecast_entsoe_transparency: 25 missing (0.05%)
BE_solar_generation_actual: 16 missing (0.03%)
BE_wind_generation_actual: 3935 missing (7.81%)
BE_wind_offshore_generation_actual: 3935 missing (7.81%)
BE_wind_onshore_generation_actual: 3 missing (0.01%)
BG_load_actual_entsoe_transparency: 65 missing (0.13%)
BG_load_forecast_entsoe_transparency: 3696 missing (7.33%)
BG_solar_generation_actual: 17 missing (0.03%)
BG_wind_onshore_generation_actual: 3 missing (0.01%)
CH_load_actual_entsoe_transparency: 3 missing (0.01%)
CH_load_forecast_entsoe_transparency: 75 missing (0.15%)
CH_solar_capacity: 6602 missing (13.10%)
CH_solar_generation_actual: 56 m

In [21]:
# For time series energy data, we use forward fill
# Why? Energy systems don't have "gaps" - if data is missing,
# the last known value is a reasonable approximation for short gaps

# Forward fill with limit (don't fill gaps > 3 hours)
df_filled = df.fillna(method='ffill', limit=3)

# For remaining gaps, use interpolation
df_filled = df_filled.interpolate(method='linear')

# Check remaining missing
print(f"\n✓ After filling: {df_filled.isnull().sum().sum()} missing values remain")

C:\Users\Kumar\AppData\Local\Temp\ipykernel_3764\2630767478.py:6: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_filled = df.fillna(method='ffill', limit=3)
C:\Users\Kumar\AppData\Local\Temp\ipykernel_3764\2630767478.py:9: FutureWarning: DataFrame.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  df_filled = df_filled.interpolate(method='linear')



✓ After filling: 750236 missing values remain


In [23]:
import numpy as np
import pandas as pd

# Work ONLY on numeric columns
numeric_cols = df_filled.select_dtypes(include="number").columns

# Check for negative values (energy values cannot be negative)
negative_check = (df_filled[numeric_cols] < 0).sum()

print("\n--- Negative Values Found ---")
print(negative_check[negative_check > 0])

# Replace negatives with NaN and interpolate safely
for col in numeric_cols:
    mask = df_filled[col] < 0
    if mask.any():
        df_filled.loc[mask, col] = np.nan
        
        # Time-aware interpolation (requires datetime index)
        df_filled[col] = df_filled[col].interpolate(method="time")

print("✓ Negative values corrected safely")



--- Negative Values Found ---
AT_price_day_ahead        376
DE_LU_price_day_ahead     484
DK_1_price_day_ahead      539
DK_2_price_day_ahead      354
GB_GBN_price_day_ahead     81
IE_sem_price_day_ahead    393
NO_1_price_day_ahead        1
NO_2_price_day_ahead        1
NO_5_price_day_ahead        1
SE_1_price_day_ahead        5
SE_2_price_day_ahead        5
SE_3_price_day_ahead        5
SE_4_price_day_ahead        5
dtype: int64
✓ Negative values corrected safely


In [24]:
# Extract time-based features for analysis
df_filled['year'] = df_filled.index.year
df_filled['month'] = df_filled.index.month
df_filled['day'] = df_filled.index.day
df_filled['hour'] = df_filled.index.hour
df_filled['dayofweek'] = df_filled.index.dayofweek  # 0=Monday, 6=Sunday
df_filled['quarter'] = df_filled.index.quarter

# Create season feature (European seasons)
def get_season(month):
    if month in [12, 1, 2]:
        return 'Winter'
    elif month in [3, 4, 5]:
        return 'Spring'
    elif month in [6, 7, 8]:
        return 'Summer'
    else:
        return 'Autumn'

df_filled['season'] = df_filled['month'].apply(get_season)

# Business hours flag (important for commercial energy analysis)
df_filled['is_business_hours'] = ((df_filled['hour'] >= 8) & 
                                   (df_filled['hour'] <= 18) & 
                                   (df_filled['dayofweek'] < 5)).astype(int)

print("✓ Derived features created")
print(f"  New columns: {df_filled.columns.tolist()}")

✓ Derived features created
  New columns: ['cet_cest_timestamp', 'AT_load_actual_entsoe_transparency', 'AT_load_forecast_entsoe_transparency', 'AT_price_day_ahead', 'AT_solar_generation_actual', 'AT_wind_onshore_generation_actual', 'BE_load_actual_entsoe_transparency', 'BE_load_forecast_entsoe_transparency', 'BE_solar_generation_actual', 'BE_wind_generation_actual', 'BE_wind_offshore_generation_actual', 'BE_wind_onshore_generation_actual', 'BG_load_actual_entsoe_transparency', 'BG_load_forecast_entsoe_transparency', 'BG_solar_generation_actual', 'BG_wind_onshore_generation_actual', 'CH_load_actual_entsoe_transparency', 'CH_load_forecast_entsoe_transparency', 'CH_solar_capacity', 'CH_solar_generation_actual', 'CH_wind_onshore_capacity', 'CH_wind_onshore_generation_actual', 'CY_load_actual_entsoe_transparency', 'CY_load_forecast_entsoe_transparency', 'CY_wind_onshore_generation_actual', 'CZ_load_actual_entsoe_transparency', 'CZ_load_forecast_entsoe_transparency', 'CZ_solar_generation_act

In [25]:
# Final validation checks
print("\n=== DATA VALIDATION SUMMARY ===")
print(f"Final shape: {df_filled.shape}")
print(f"Missing values: {df_filled.isnull().sum().sum()}")
print(f"Duplicate timestamps: {df_filled.index.duplicated().sum()}")
print(f"Date range: {df_filled.index.min()} to {df_filled.index.max()}")

# Check value ranges are sensible
print("\n--- Value Range Checks ---")
for col in ['consumption', 'wind', 'solar', 'price']:
    if col in df_filled.columns:
        print(f"{col}: min={df_filled[col].min():.2f}, max={df_filled[col].max():.2f}")


=== DATA VALIDATION SUMMARY ===
Final shape: (50401, 307)
Missing values: 750236
Duplicate timestamps: 0
Date range: 2014-12-31 23:00:00+00:00 to 2020-09-30 23:00:00+00:00

--- Value Range Checks ---


In [26]:
# Save to processed folder
df_filled.to_csv('../data/processed/energy_data_cleaned.csv')

print("\n✓ Cleaned data saved to data/processed/energy_data_cleaned.csv")
print(f"  Rows: {len(df_filled)}")
print(f"  Columns: {len(df_filled.columns)}")


✓ Cleaned data saved to data/processed/energy_data_cleaned.csv
  Rows: 50401
  Columns: 307
